In [1]:
import glob
import sys
import copy
import math
import scipy as sp
from scipy import ndimage as ndi
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.cm as cm

from skimage import io
from skimage.external.tifffile import imread as tifimr
#import skimage.external.tifffile as tifim
from skimage.measure import label, regionprops
from skimage.morphology import closing,square,remove_small_holes

import cv2

from sklearn import svm


# -------------------------------------------- API
import requests                           # NEED to be installed
import json                               # Python standard library

# customized library by Zachary Trautt
import libs.mdcs as mdcs
# customized library by Lucas Hale
import libs.DataModelDict as DMD

%matplotlib inline

In [2]:
USER = "     "
PSWD = "XXXXX"
MDCS_URL = "CCCC"

## query images from MDCS

In [3]:
schema_title = 'SemImage'

url = MDCS_URL + "/rest/templates/select/all"

allSchemas = json.loads(requests.get(url, auth=(USER, PSWD)).text)
schemaId = []
for i in range(len(allSchemas)):
    if allSchemas[i]['title'] == schema_title:
        schemaId.append(allSchemas[i]['id'])

        
url = MDCS_URL + "/rest/explore/query-by-example"
query = {"schema":schemaId[0]}
req_data = {"query":json.dumps(query)}
qres = json.loads(requests.post(url,req_data,auth=(USER, PSWD)).text)

imgfile = []
img_urls = []


for i in range(len(qres)):
    imgfile.append(qres[i]['title'])
    content = qres[i]['content']
    qdata = DMD.DataModelDict(content)
    img_urls.append(qdata.find('imageFile'))
    

print "no_images: ",len(img_urls)
print
print imgfile
print
print img_urls
    

no_images:  9

[u'1045_Steel_Nital-etch-01', u'1045_Steel_Nital-etch-02', u'1045_Steel_Nital-etch-03', u'1045_Steel_Nital-etch-04', u'1045_Steel_Nital-etch-05', u'1045_Steel_Nital-etch-06', u'1045_Steel_Nital-etch-07', u'20150911_1045_Nital_etch', u'20150911_1045_Nital_etch2']

[u'http://129.6.153.123:8000/rest/blob?id=57d98eead1ed024826b93f61', u'http://129.6.153.123:8000/rest/blob?id=57d98eebd1ed024826b93f68', u'http://129.6.153.123:8000/rest/blob?id=57d98eebd1ed024826b93f6f', u'http://129.6.153.123:8000/rest/blob?id=57d98eecd1ed024826b93f76', u'http://129.6.153.123:8000/rest/blob?id=57d98eecd1ed024826b93f7d', u'http://129.6.153.123:8000/rest/blob?id=57d98eedd1ed024826b93f84', u'http://129.6.153.123:8000/rest/blob?id=57d98eedd1ed024826b93f8b', u'http://129.6.153.123:8000/rest/blob?id=57d98eeed1ed024826b93f92', u'http://129.6.153.123:8000/rest/blob?id=57d98eefd1ed024826b93fa3']


## Image analysis

In [5]:
for img_url in img_urls:

    openfile = io.imread(img_url)

    if len(openfile) == 3:                                 # (3, 1024, 1280)  -- original .tif
        SEM = np.copy(openfile[0])

    elif(len(openfile) > 3 and len(openfile.shape) == 3):  # (1024, 1280, 3)  -- the oother codec
        SEM = np.zeros((len(openfile),len(openfile[0])))

        for i in range(len(openfile)):
            for j in range(len(openfile[0])):
                SEM[i][j] = int((openfile[i][j][0]+1)**2-1)

    elif(len(openfile.shape) == 2):
        SEM = np.asarray(openfile)
    else:
        print "file format does not match"

    SEM = np.asarray(SEM)

    '''
    fig, ax = plt.subplots(figsize=(16,12))
    ax.imshow(SEM, cmap=plt.cm.gray)
    ax.axis('off')
    plt.show()
    '''

    dim = np.zeros(2,dtype=int)
    dim[0], dim[1] = SEM.shape

    b_line_finish = 0
    min_intensity = np.average(SEM[-1,:])

    for i in range(-1,-int(0.3*dim[0]),-1):
        if(np.average(SEM[i,:]) == min_intensity):
            b_line_finish = i


    # microstructure
    pic_micro = SEM[0:b_line_finish]


    dim_micro = np.zeros(2,dtype=int)
    dim_micro[0], dim_micro[1] = pic_micro.shape

    info_bar_y = int(dim[0]/1024)
    info_bar_x = int(dim[1]/1024)

    # information
    pic_scale = SEM[dim[0]+b_line_finish:dim[0]]


    label_img = label(pic_scale,background=0)
    regions = regionprops(label_img)
    props = regions[1]

    y0, x0 = props.centroid

    min_boundary = np.zeros(2,dtype=int)
    max_boundary = np.zeros(2,dtype=int)
    min_boundary[1], min_boundary[0], max_boundary[1], max_boundary[0] = props.bbox
    bx = (min_boundary[0], max_boundary[0],
          max_boundary[0], min_boundary[0], min_boundary[0])
    by = (min_boundary[1], min_boundary[1],
          max_boundary[1], max_boundary[1], min_boundary[1])

    diff = np.zeros(2,dtype=float)
    for i_bound in range(2):
        diff[i_bound] = max_boundary[i_bound] - min_boundary[i_bound]

    unit_pixel = int(max(diff))


    #fig, ax = plt.subplots(figsize=(16,12))
    #ax.plot(x0, y0, '.g', markersize=5)
    #ax.plot(bx, by, '-b', linewidth=2.5)


    label_img = label(pic_scale,connectivity=1,neighbors=8)
    regions = regionprops(label_img)

    no_region = 0
    unit_ = []


    # ==== Machine Learning
    refimgs = 'libs/refimg/' + str(dim[0]) + '/*.tif'
    files = glob.glob(refimgs)

    read_img = []
    ref_value = []
    for file_ in files:
        read_img.append(io.imread(file_).flatten())
        ref_value.append(file_[file_.rfind('/')+1:file_.index('_')])

    ref_img = np.zeros((len(ref_value),2500))
    for i in range(len(read_img)):
        ref_img[i][:len(read_img[i])] = read_img[i]

    clf = svm.SVC()
    clf.fit(ref_img,ref_value)


    for props in regionprops(label_img):
        y0, x0 = props.centroid

        if((7*info_bar_y)**2 < props.area < (12**info_bar_y)**2 and
           0 < y0 < 20*info_bar_y and
           800*info_bar_x < x0 < 950*info_bar_x):

            region_no = str(no_region)
            #ax.text(x0, y0, region_no, fontsize=20, color='red', horizontalalignment='center',
                    #bbox={'facecolor':'white', 'alpha':0.5, 'pad':10})

            #ax.plot(x0, y0, '.g', markersize=5)

            min_boundary = np.zeros(2,dtype=int)
            max_boundary = np.zeros(2,dtype=int)
            min_boundary[1], min_boundary[0], max_boundary[1], max_boundary[0] = props.bbox

            '''
            bx = (min_boundary[0], max_boundary[0], max_boundary[0], min_boundary[0], min_boundary[0])
            by = (min_boundary[1], min_boundary[1], max_boundary[1], max_boundary[1], min_boundary[1])
            ax.plot(bx, by, '-b', linewidth=2.5)
            '''        

            ylb = min_boundary[1]
            yub = max_boundary[1]
            xlb = min_boundary[0]
            xub = max_boundary[0]

            char_ = [] 
            for i in range(ylb,yub):
                char_.append(pic_scale[i][xlb:xub])

            char_ = np.asarray(char_).flatten()

            char = np.zeros((2500))
            char[:len(char_)] = char_
            unit_.append([x0,clf.predict(char.reshape(1,-1))[0]])

        no_region += 1

    unit_ = np.array(sorted(unit_))
    unitlist = np.array(unit_[:,1])

    '''
    plt.xlim(800*info_bar_x, 950*info_bar_x)

    ax.imshow(pic_scale, cmap=plt.cm.gray)
    plt.axis('off')
    plt.show()
    '''


    unit_length = ''
    for i in range(len(unitlist)-2):
        unit_length += str(unitlist[i])


    unit_length = float(unit_length)
    unit_unitlength = str(unitlist[-2]) + str(unitlist[-1])

    #print 'Pixels',unit_pixel,': ',unit_length,'__',unit_unitlength
    hole_length = 0.2  # micrometer
    if(unit_unitlength == 'micrometer'):
        fill_hole_pixels = (hole_length * unit_pixel / unit_length) ** 2.

    #print fill_hole_pixels

    # -------------------------------------------------------
    freq, bins = np.histogram(pic_micro, bins=20)
    freq_sorted = sorted(freq, reverse=True)

    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2

    mx = (np.median(pic_micro), np.median(pic_micro))
    my = (np.amax(freq),0)

    max_intensity = max(my)
    min_intensity = min(my)

    '''
    fig, ax = plt.subplots(figsize=(12,9))
    plt.bar(center, freq, align='center', width=width)
    ax.plot(mx, my, '-r', linewidth=2.5)
    plt.show()
    '''

    # --------------------------------- increase contrast
    washed_micro = np.copy(pic_micro)

    binary_threshold = 6

    washed_micro[pic_micro <= bins[binary_threshold]] = min_intensity
    washed_micro[pic_micro > bins[binary_threshold]] = max_intensity

    tot_pixel = len(pic_micro[:,0])*len(pic_micro[0,:])

    '''
    fig, ax = plt.subplots(figsize=(16,12))
    plt.axis('off')
    ax.imshow(washed_micro, cmap=plt.cm.gray)
    '''

    # --------------------------------- clean noise
    label_objects, nb_labels = ndi.label(washed_micro)

    sizes = np.bincount(label_objects.ravel())
    mask_sizes = sizes > fill_hole_pixels
    mask_sizes[0] = min_intensity
    map_cleaned = mask_sizes[label_objects]

    '''
    fig, ax = plt.subplots(figsize=(16,12))
    ax.imshow(map_cleaned, cmap=plt.cm.gray)
    ax.axis('off')
    plt.show()
    '''

    # --------------------------------- fill hole
    map_cleaned2 = ndi.binary_closing(map_cleaned, square(5))

    image = np.copy(map_cleaned2)
    rsh = remove_small_holes(image, 1000).astype(int)
    white_region = np.mean(rsh)
    black_region = 1. - white_region

    '''
    fig, ax = plt.subplots(figsize=(16,12))
    ax.imshow(rsh, cmap=plt.cm.gray)
    ax.axis('off')
    plt.show()
    '''

    image = np.copy(rsh.astype(int))
    label_img = label(image,background=1)

    '''
    fig, ax = plt.subplots(figsize=(16,12))
    ax.imshow(image, cmap=plt.cm.gray)
    '''
    
    # ========================================= Stastics
    df_temp = pd.DataFrame(columns=['tempvalue'])
    
    for props in regionprops(label_img):
        if 1000 < props.area < 0.2*tot_pixel:
            df_temp.loc[len(df_temp)] = [props.equivalent_diameter]

            
    headers = ['volume fraction','no_test_grains','eq_diameter, '+unit_unitlength, 'STD']
    df_grainsize = pd.DataFrame(columns=headers)

    
    if len(df_temp) > 0:
        equivalent_diameter = df_temp['tempvalue'].mean() * unit_length / unit_pixel
        if len(df_temp) > 1:
            eq_dia_std = df_temp.std()[0] * unit_length / unit_pixel
        else:
            eq_dia_std = 0

        
        df_grainsize.loc['Ferrite'] = [black_region,len(df_temp),equivalent_diameter,eq_dia_std]

        #plt.axis('off')
        #plt.show()

        #print df_grainsize

    else:
        #print "Not able to detect the white regions from image"
        #print

        df_grainsize.loc['Ferrite'] = [black_region,'0','0','0']
        #print df_grainsize


    image = np.copy(rsh.astype(int))

    label_img = label(image,background=0)

    '''
    fig, ax = plt.subplots(figsize=(16,12))
    ax.imshow(image, cmap=plt.cm.gray)
    '''

    df_temp = pd.DataFrame(columns=['tempvalue'])
    for props in regionprops(label_img):
        if 1000 < props.area < 0.2*tot_pixel:
            df_temp.loc[len(df_temp)] = [props.equivalent_diameter]

    if len(df_temp) > 0:
        equivalent_diameter = df_temp['tempvalue'].mean() * unit_length / unit_pixel
        if len(df_temp) > 1:
            eq_dia_std = df_temp.std()[0] * unit_length / unit_pixel
        else:
            eq_dia_std = 0

        df_grainsize.loc['Pearlite'] = [white_region,len(df_temp),equivalent_diameter,eq_dia_std]

        
        #print df_grainsize
    else:
        #print "Not able to detect the white regions from image"
        #print

        df_grainsize.loc['Pearlite'] = [white_region,'0','0','0']
        #print df_grainsize

    img = np.uint8(map_cleaned)
    mean_intensity_micro = np.mean(map_cleaned)

    test_radius = int(unit_pixel/5)
    test_r2 = int(math.pow(test_radius,2))
    test_points = 120

    lbx = test_radius + 2
    ubx = dim_micro[1] - lbx
    lby = test_radius + 2
    uby = dim_micro[0] - lby

    headers = ['center_x','center_y','no_cross_section','mean_random_spacing, ' + unit_unitlength]
    df_meanspacing = pd.DataFrame(columns=headers)

    no_centers = 200
    cy = np.random.random_integers(lby,high=uby,size=no_centers)
    cx = np.random.random_integers(lbx,high=ubx,size=no_centers)

    angle = np.linspace(0,2.*np.pi,num=test_points,endpoint=False)
    white_region2 = 0.

    #fig, ax = plt.subplots(figsize=(12,9))


    for i in range(no_centers):

        mean_intensity = np.mean(img[cy[i]-test_radius:cy[i]
                                     +test_radius,cx[i]-test_radius:cx[i]
                                     +test_radius])
        #print mean_intensity,mean_intensity_micro

        if(mean_intensity >= mean_intensity_micro):
            test_circle = []

            # buffers
            no_crosssections = 0
            intensity = 0
            intensity_p = 0

            x = cx[i] + int(test_radius*np.cos(0.))
            y = cy[i] + int(test_radius*np.sin(0.))

            intensity = img[y,x]
            if(intensity == 1):
                white_region2 += 1/test_points

            intensity_p = intensity

            for j in angle[1:]:
                x = cx[i] + int(test_radius*np.cos(j))
                y = cy[i] + int(test_radius*np.sin(j))

                intensity = img[y,x]

                if(intensity == 1):
                    white_region2 += (1./test_points)/no_centers

                if(intensity + intensity_p == 1):
                    #ax.scatter(x,y,s=10,c='r')
                    no_crosssections += 1

                intensity_p = intensity
                test_circle.append([x,y])

            test_circle.sort()
            test_circle = np.asarray(test_circle)

            if no_crosssections > 0:
                random_spacing = (np.pi * test_radius 
                                  * (unit_length/unit_pixel)
                                  / no_crosssections)
            else:
                random_spacing = 0.

            df_meanspacing.loc[len(df_meanspacing)] = [cx[i],cy[i],no_crosssections,random_spacing]

            #ax.scatter(cx[i],cy[i],s=50,c='r')
            #ax.scatter(test_circle[:,0],test_circle[:,1],s=5,c='b')


    '''
    ax.axis('off')
    ax.imshow(img, cmap=plt.cm.gray)
    plt.show()
    '''

    mean_spacing = df_meanspacing[headers[-1]].mean()

    if len(df_meanspacing) > 1:
        mean_spacing_std = df_meanspacing.std()[-1]
    else:
        mean_spacing_std = 0
    
    df_grainsize.loc['Cementite'] = [white_region2*white_region,len(df_meanspacing)
                                     ,mean_spacing,mean_spacing_std]
    
    print '-----------------------------------------------------------------------------'
    print imgfile[img_urls.index(img_url)]
    print df_grainsize
    

-----------------------------------------------------------------------------
1045_Steel_Nital-etch-01
           volume fraction no_test_grains eq_diameter, micrometer        STD
Ferrite           0.316783             59                 6.96185    5.03334
Pearlite          0.683217              0                       0          0
Cementite         0.189223            108                0.209759  0.0620948
-----------------------------------------------------------------------------
1045_Steel_Nital-etch-02
           volume fraction no_test_grains eq_diameter, micrometer        STD
Ferrite           0.089487             30                 3.69832    2.11567
Pearlite          0.910513              0                       0          0
Cementite         0.437539            124                0.302331  0.0948977
-----------------------------------------------------------------------------
1045_Steel_Nital-etch-03
           volume fraction no_test_grains eq_diameter, micrometer       STD